In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from datetime import datetime, date, time

import gc
import copy

import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, KFold

In [3]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=1234
pl.seed_everything(random_seed)

Global seed set to 1234


1234

In [10]:

id_feats = ['customer_ID']
#date_col =  'S_2'
target_col = 'target'


x_cols = ['B_10__last', 'B_10__mean', 'B_11__last', 'B_11__last__log', 'B_11__mean', 'B_11__mean__log', 'B_11__min', 'B_12__last', 'B_12__last__log', 'B_13__last', 'B_13__last__log', 'B_14__last', 'B_14__min', 'B_15__last', 'B_15__mean', 'B_16__last', 'B_16__max', 'B_16__min', 'B_17__last', 'B_17__max', 'B_17__mean', 'B_17__mean2std', 'B_17__min', 'B_18__last', 'B_18__last__log', 'B_18__mean__log', 'B_18__min', 'B_19__last', 'B_19__min', 'B_1__last', 'B_1__max', 'B_1__mean', 'B_1__min', 'B_20__last', 'B_20__max', 'B_20__mean', 'B_20__min', 'B_21__last', 'B_21__last__log', 'B_21__max', 'B_21__mean__log', 'B_22__last', 'B_22__last__log', 'B_22__max', 'B_22__mean', 'B_22__mean__log', 'B_22__min', 'B_23__last', 'B_23__last__log', 'B_23__mean', 'B_23__min', 'B_24__last', 'B_24__last__log', 'B_24__mean', 'B_24__mean__log', 'B_25__last', 'B_25__min', 'B_26__last', 'B_26__last__log', 'B_26__mean__log', 'B_27__last', 'B_27__last__log', 'B_28__last', 'B_29__last', 'B_29__last__log', 'B_2__last', 'B_2__mean', 'B_2__min', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_30__last', 'B_30__nunique', 'B_31__last', 'B_31__nunique', 'B_32__last', 'B_32__last__log', 'B_32__max', 'B_33__last', 'B_33__max', 'B_33__mean', 'B_33__min', 'B_36__last', 'B_36__last__log', 'B_37__last', 'B_37__max', 'B_37__mean', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_38__last', 'B_38__nunique', 'B_39__last', 'B_3__last', 'B_3__last__log', 'B_3__max', 'B_3__mean', 'B_3__mean__log', 'B_3__min', 'B_40__last', 'B_40__last__log', 'B_40__mean__log', 'B_40__min', 'B_41__last', 'B_41__last__log', 'B_42__last', 'B_42__last__log', 'B_4__last', 'B_4__last__log', 'B_4__max', 'B_4__mean2std', 'B_4__mean__log', 'B_5__last', 'B_5__last__log', 'B_5__mean', 'B_6__last', 'B_6__min', 'B_7__last', 'B_7__max', 'B_7__mean', 'B_7__min', 'B_8__last', 'B_8__max', 'B_8__mean', 'B_8__min', 'B_9__last', 'B_9__last__log', 'B_9__max', 'B_9__mean', 'B_9__mean__log', 'B_9__min', 'D_102__last', 'D_102__last__log', 'D_102__max', 'D_103__last', 'D_104__last', 'D_105__last', 'D_105__max', 'D_106__last', 'D_106__last__log', 'D_107__last', 'D_107__last__log', 'D_107__max', 'D_107__mean__log', 'D_108__last', 'D_108__last__log', 'D_109__last', 'D_109__last__log', 'D_109__mean', 'D_110__last', 'D_111__last', 'D_112__last', 'D_113__last', 'D_113__last__log', 'D_113__max', 'D_114=0.0', 'D_114__last', 'D_114__nunique', 'D_115__last', 'D_115__last__log', 'D_116__last', 'D_117__last', 'D_118__last', 'D_118__last__log', 'D_118__mean__log', 'D_119__last', 'D_119__last__log', 'D_120=1.0', 'D_120__last', 'D_120__nunique', 'D_121__last', 'D_122__last', 'D_122__max', 'D_122__min', 'D_123__last', 'D_123__last__log', 'D_123__mean', 'D_124__last', 'D_125__last', 'D_125__last__log', 'D_126__last', 'D_127__last', 'D_128__last', 'D_128__max', 'D_128__min', 'D_129__last', 'D_129__max', 'D_129__mean', 'D_130__last', 'D_130__max', 'D_131__last', 'D_131__last__log', 'D_131__max', 'D_131__min', 'D_132__last', 'D_132__mean2std', 'D_132__min', 'D_133__last', 'D_133__last__log', 'D_133__max', 'D_133__min', 'D_134__last', 'D_134__min', 'D_135__last', 'D_135__last__log', 'D_135__mean2std', 'D_136__last', 'D_136__last__log', 'D_137__last', 'D_137__last__log', 'D_138__last', 'D_138__last__log', 'D_139__last', 'D_139__mean', 'D_140__last', 'D_140__last__log', 'D_140__max', 'D_141__last', 'D_142__last', 'D_143__last', 'D_144__last', 'D_145__last', 'D_39__last', 'D_39__last__log', 'D_39__max', 'D_39__mean', 'D_41__last', 'D_41__last__log', 'D_41__max', 'D_41__mean', 'D_41__min', 'D_42__last', 'D_42__max', 'D_42__mean', 'D_42__mean2std', 'D_42__min', 'D_43__last', 'D_43__last__log', 'D_43__max', 'D_43__mean', 'D_43__mean__log', 'D_43__min', 'D_44__last', 'D_44__last__log', 'D_44__max', 'D_44__mean', 'D_44__mean__log', 'D_44__min', 'D_45__last', 'D_45__last__log', 'D_45__max', 'D_45__mean', 'D_45__mean__log', 'D_45__min', 'D_46__last', 'D_46__mean', 'D_46__mean2std', 'D_46__min', 'D_47__last', 'D_48__last', 'D_48__max', 'D_48__mean', 'D_48__min', 'D_49__last', 'D_49__last__log', 'D_49__mean2std', 'D_50__last', 'D_51__last', 'D_51__last__log', 'D_51__mean', 'D_52__last', 'D_52__max', 'D_52__mean', 'D_52__mean2std', 'D_52__min', 'D_53__last', 'D_53__max', 'D_53__mean2std', 'D_53__min', 'D_54__last', 'D_55__last', 'D_55__min', 'D_56__last', 'D_56__min', 'D_58__last', 'D_58__min', 'D_59__last', 'D_59__max', 'D_60__last', 'D_60__last__log', 'D_60__max', 'D_61__last', 'D_61__max', 'D_61__mean', 'D_61__min', 'D_62__last', 'D_62__max', 'D_62__mean', 'D_62__min', 'D_64=U', 'D_65__last', 'D_65__max', 'D_65__mean', 'D_68=1.0', 'D_68__last', 'D_69__last', 'D_69__min', 'D_70__last', 'D_70__max', 'D_70__mean2std', 'D_70__min', 'D_71__last', 'D_72__last', 'D_72__max', 'D_72__min', 'D_73__last', 'D_74__last', 'D_74__max', 'D_74__mean', 'D_75__last', 'D_75__max', 'D_75__mean', 'D_76__last', 'D_77__last', 'D_77__max', 'D_77__mean', 'D_77__min', 'D_78__last', 'D_78__max', 'D_78__mean', 'D_79__last', 'D_79__max', 'D_80__last', 'D_81__last', 'D_81__max', 'D_81__mean', 'D_82__last', 'D_83__last', 'D_84__last', 'D_84__max', 'D_84__mean', 'D_86__last', 'D_87__last', 'D_88__last', 'D_89__last', 'D_89__max', 'D_89__mean', 'D_91__last', 'D_92__last', 'D_93__last', 'D_94__last', 'D_96__last', 'P_2__last', 'P_2__max', 'P_2__mean', 'P_2__min', 'P_3__last', 'P_3__max', 'P_3__mean', 'P_3__min', 'P_4__last', 'P_4__max', 'P_4__mean', 'P_4__min', 'R_10__last', 'R_10__max', 'R_10__mean', 'R_11__last', 'R_11__max', 'R_11__mean', 'R_12__last', 'R_12__max', 'R_12__mean', 'R_13__last', 'R_13__max', 'R_13__mean', 'R_13__mean2std', 'R_14__last', 'R_15__last', 'R_15__max', 'R_15__mean', 'R_16__last', 'R_16__max', 'R_16__mean', 'R_17__last', 'R_17__max', 'R_18__last', 'R_18__max', 'R_19__last', 'R_19__min', 'R_1__last', 'R_1__max', 'R_1__mean', 'R_1__min', 'R_20__last', 'R_20__max', 'R_20__mean', 'R_21__last', 'R_22__last', 'R_22__max', 'R_23__last', 'R_24__last', 'R_24__max', 'R_24__mean', 'R_25__last', 'R_25__max', 'R_26__last', 'R_26__mean2std', 'R_27__last', 'R_27__max', 'R_27__mean', 'R_27__min', 'R_28__last', 'R_28__last__log', 'R_2__last', 'R_2__max', 'R_2__mean', 'R_2__min', 'R_3__last', 'R_3__max', 'R_3__mean', 'R_3__min', 'R_4__last', 'R_4__max', 'R_4__mean', 'R_4__min', 'R_5__last', 'R_5__max', 'R_5__mean', 'R_6__last', 'R_6__max', 'R_6__mean', 'R_7__last', 'R_7__max', 'R_7__mean', 'R_8__last', 'R_8__max', 'R_8__mean', 'R_9__last', 'R_9__mean2std', 'S_11__last', 'S_11__mean', 'S_12__last', 'S_12__max', 'S_13__last', 'S_15__last', 'S_15__max', 'S_15__mean', 'S_16__last', 'S_17__last', 'S_17__min', 'S_18__last', 'S_19__last', 'S_20__last', 'S_20__max', 'S_22__last', 'S_22__max', 'S_22__mean', 'S_23__last', 'S_23__max', 'S_23__mean', 'S_24__last', 'S_25__last', 'S_25__mean', 'S_25__min', 'S_26__last', 'S_26__last__log', 'S_27__last', 'S_27__mean2std', 'S_3__last', 'S_3__max', 'S_3__mean', 'S_3__min', 'S_5__last', 'S_5__last__log', 'S_5__mean__log', 'S_6__last', 'S_6__max', 'S_6__min', 'S_7__last', 'S_7__max', 'S_7__mean', 'S_7__min', 'S_8__last', 'S_8__mean', 'S_8__min', 'S_9__last', 'S_9__max']


len(x_cols)

489

In [ ]:
%%time
# train_file = r'/kaggle/input/amex-agg-data-rev2/agg_train_all_rev2_rev.parquet'
train_file = r'amex/agg_v3/agg_train_all_small.parquet'
df = pd.read_parquet(train_file, columns=id_feats + [target_col] + x_cols, engine='pyarrow')


## hyperopt parameters

In [ ]:
learn_rates = np.concatenate((
#                            np.arange(0.00001, 0.0001, 0.00001),  
                           np.arange(0.0001, 0.001, 0.0001), 
                           np.arange(0.001, 0.01, 0.001), 
                           np.arange(0.01, 0.1, 0.01)
                          ), 
                          axis=0)


len(learn_rates)

In [ ]:
#https://lightgbm.readthedocs.io/en/latest/Parameters.html
#https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html?highlight=classifier#lightgbm.LGBMClassifier

from hyperopt import hp
import numpy as np
space  = { 
    
                 'n_estimators': hp.choice('n_estimators', range(500, 2000, 50)),#num_boost_round
                 'boosting_type':hp.choice('boosting_type', ['gbdt']),#boosting
                 'objective':hp.choice('objective', ['binary'] ),
                 'metric':hp.choice('metric', ['auc']),
                 'learning_rate':  hp.choice('learning_rate', learn_rates), 
                 'colsample_bytree': hp.choice('colsample_bytree', np.round(np.arange(0.1, 0.86, 0.05),3)), #feature_fraction
                 'max_depth': hp.choice('max_depth', range(7, 36, 1)), #int type
                 'min_child_samples': hp.choice('min_child_samples',range(100, 5001, 50)), #min_data_in_leaf
                 'reg_alpha':hp.choice('reg_alpha', [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,15]),#lambda_l1
                 'reg_lambda':hp.choice('reg_lambda', [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,15]),#lambda_l2
                 'max_bin':hp.choice('max_bin', range(500, 10000, 50)),
                 'min_data_in_bin':hp.choice('min_data_in_bin', range(500, 9000, 50)),
                 'subsample':hp.choice('subsample', np.round(np.arange(0.1, 0.96, 0.05),3)), #bagging_fraction
                 'subsample_freq':hp.choice('subsample_freq', range(1, 100, 2)),#bagging_freq
                 #max number of leaves in one tree. 1 < num_leaves <= 131072. classes< num_leaves< 2^max_depth  
                 'num_leaves':hp.choice('num_leaves', range(31, 300, 5)),#max_leaves. 
                 'random_state':hp.choice('random_state', [1234]),
                 'n_jobs':hp.choice('n_jobs', [4]),#nthread
                 #'min_split_gain':hp.choice('min_split_gain', [0.0]), #min_gain_to_split
                 #'min_child_weight':hp.choice('min_child_weight', [0.001]),   #min_sum_hessian_in_leaf
                 #'subsample_for_bin':hp.choice('subsample_for_bin', [200000]),   #bin_construct_sample_cnt 
                 #'importance_type':hp.choice('importance_type', ['split']),   
    
                  }
                  
    


### lightgbm

In [ ]:
import lightgbm as lgb
def train_trees(X_train, y_train, num_round=100, params={} ):
    
    params['verbosity'] = -1
    dtrain = lgb.Dataset(X_train, y_train)
    
    tree_model = lgb.train(params,
                dtrain,
                num_boost_round=num_round)
    
    
    del dtrain
    gc.collect()

    return tree_model

In [ ]:
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)


In [ ]:
X = df[x_cols]
y = df[['target']]

In [ ]:
del df
gc.collect()

In [ ]:
skf = KFold(n_splits=3)

In [ ]:
print(skf)

for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     print("TRAIN:", len(train_index), "TEST:", len(test_index), len(test_index)/(len(test_index)+len(train_index)))
    print(y.iloc[test_index]['target'].value_counts()/len(test_index)) 
    print(y.iloc[train_index]['target'].value_counts()/len(train_index))

In [ ]:
len(train_index), len(test_index)

In [ ]:
log_file = 'amex/agg_v3/lgb-hyperopt-v3data-3kfold-489feats.xlsx'

In [ ]:
loss_dict = []


def score(params):
#     print(params)
    num_boost_round = params['n_estimators']
    params_ = copy.deepcopy(params)
    del params_['n_estimators']
    
    losses = []
    
    for train_index, test_index in skf.split(X, y):
        
        #----start: data prep-------------------------------------
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #----end: data prep-------------------------------------
        
        #-----start: train trees---------------------------------------
        tree_model = train_trees(X_train, y_train['target'].values,
                                 num_round=num_boost_round, 
                                 params=params_
                                )

        y_preds = tree_model.predict(X_test, num_iteration=tree_model.best_iteration)
#         loss = roc_auc_score(y_test, y_preds)
#         loss = f1_score(y_test['target'].values, (y_preds>=0.9).astype(int))
#         loss = amex_metric(y_test, 
#                            pd.DataFrame(data={'prediction': y_preds}))
        loss = amex_metric(y_test['target'].values, y_preds)
        losses.append(loss)
        #-----end: train trees---------------------------------------
        
    loss = np.mean(losses)
#     print(loss)
    loss_dict.append({'params': params, 'losses': losses, 'mean_loss': loss})
    
    if len(loss_dict)%10==0:
        pd.DataFrame(data=loss_dict).to_excel(log_file, index=False)
    return {'loss': -loss, 'status': STATUS_OK}

In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, anneal, rand
from functools import partial
def optimize(space, evals, cores, trials, optimizer=tpe.suggest, random_state=1234, n_startup_jobs=10):
    algo = partial(optimizer, n_startup_jobs=n_startup_jobs)
    best = fmin(score, space, algo=algo, max_evals=evals, trials = trials)
    print(best)
    return best

In [ ]:
cores = 4
n=500
verbose = False
trials = Trials()

In [ ]:
best_param = optimize(space,
                      evals = n,
                      optimizer=tpe.suggest,
                      cores = cores,
                      trials = trials, random_state=1234, 
                      n_startup_jobs=10)

In [ ]:
pd.DataFrame(data=loss_dict).to_excel(log_file, index=False)